In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plot
import seaborn
import re

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy.stats as stats

In [2]:
# Configure how graphs will show up in this notebook
%matplotlib inline
seaborn.set_context('notebook', rc={'figure.figsize': (50, 50)}, font_scale=1.5)

In [3]:
%%time
def load_embeddings(filename):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        for i, line in enumerate(infile):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype='f')

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 6.2 µs


In [5]:
%%time 
glove_840B_embeddings = load_embeddings('data/embeddings/Glove/glove.840B.300d.txt')
glove_840B_embeddings.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data/embeddings/Glove/glove.840B.300d.txt'

In [ ]:
%%time 
glove_6B_300d_embeddings = load_embeddings('data/embeddings/Glove/glove.6B.300d.txt')
glove_6B_300d_embeddings.shape

In [ ]:
## Loading word2vec
from conceptnet5.vectors.formats import load_word2vec_bin
w2v = load_word2vec_bin('large_data/word2vec-googlenews-300.bin.gz', nrows=2000000)

# word2vec is case-sensitive, so case-fold its labels
w2v.index = [label.casefold() for label in w2v.index]

# Now we have duplicate labels, so drop the later (lower-frequency) occurrences of the same label
w2v = w2v.reset_index().drop_duplicates(subset='index', keep='first').set_index('index')

In [ ]:
def load_lexicon(filename):
    """
    Load a file from Bing Liu's sentiment lexicon
    (https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), containing
    English words in Latin-1 encoding.
    
    One file contains a list of positive words, and the other contains
    a list of negative words. The files contain comment lines starting
    with ';' and blank lines, which should be skipped.
    """
    lexicon = []
    with open(filename, encoding='latin-1') as infile:
        for line in infile:
            line = line.rstrip()
            if line and not line.startswith(';'):
                lexicon.append(line)
    return lexicon

In [ ]:
pos_words = load_lexicon('data/positive-words.txt')
neg_words = load_lexicon('data/negative-words.txt')

print(len(pos_words), len(neg_words))

In [ ]:
import re
TOKEN_RE = re.compile(r"\w.*?\b")

def vecs_to_sentiment(model, vecs):
    # predict_log_proba gives the log probability for each class
    predictions = model.predict_log_proba(vecs)

    # To see an overall positive vs. negative classification in one number,
    # we take the log probability of positive sentiment minus the log
    # probability of negative sentiment.
    # this is a logarithm of the max margin for the classifier, 
    # similar to odds ratio (but not exact) log(p_1/p_0) = log(p_1)-log(p_0)
    return predictions[:, 1] - predictions[:, 0]


def words_to_sentiment(model, embeddings, words):
    vecs = embeddings.loc[embeddings.index.intersection(set(words))].dropna()
    # vecs = embeddings.loc[words].dropna()
    log_odds = vecs_to_sentiment(model, vecs)
    return pd.DataFrame({'sentiment': log_odds}, index=vecs.index)
    

def text_to_sentiment(model, embeddings, text):
    # tokenize the input phrase
    tokens = [token.casefold() for token in TOKEN_RE.findall(text)]
    # send each token separately into the embedding, then the classifier
    sentiments = words_to_sentiment(model, embeddings, tokens)
    return sentiments['sentiment'].mean() # return the mean for the classifier        

In [ ]:
def texts_to_sentiment(model, embeddings, texts):
    sentiments = pd.DataFrame(columns=['sentiment'])
    for text in texts:
        sentiments= pd.concat([sentiments, pd.DataFrame.from_records([{'sentiment': text_to_sentiment(model, embeddings, text)}])])
    return sentiments


In [ ]:
def category_sentiment_table(model, embeddings, group_data, isText=False):
    frames = []
    for group, name_list in sorted(group_data.items()):
        lower_names = [name.lower() for name in name_list]
        if isText:
            sentiments = texts_to_sentiment(model, embeddings, lower_names)
        else:
            sentiments = words_to_sentiment(model, embeddings, lower_names)
        sentiments['group'] = group
        frames.append(sentiments)

    # Put together the data we got from each ethnic group into one big table
    return pd.concat(frames)

In [ ]:
def train_model(embeddings, title = ''):
    # get the positive and negative embeddings
    pos_words_common = list(set(pos_words) & set(embeddings.index)) 
    neg_words_common = list(set(neg_words) & set(embeddings.index)) 
    pos_vectors = embeddings.loc[pos_words_common]
    neg_vectors = embeddings.loc[neg_words_common]
    vectors = pd.concat([pos_vectors, neg_vectors])
    targets = np.array([1 for entry in pos_vectors.index] + [-1 for entry in neg_vectors.index])
    labels = list(pos_vectors.index) + list(neg_vectors.index)

    # split the data
    train_vectors, test_vectors, train_targets, test_targets, train_labels, test_labels = \
        train_test_split(vectors, targets, labels, test_size=0.1, random_state=0)
    
    # train our model
    model = SGDClassifier(loss='log_loss', random_state=0, max_iter=100)
    model.fit(train_vectors, train_targets)
    
    # print out a goodness of fit
    accuracy = accuracy_score(model.predict(test_vectors), test_targets)
    print("Accuracy of sentiment: {:.2%}".format(accuracy))
    return model

In [ ]:
model_6B = train_model(glove_6B_300d_embeddings)

In [ ]:
model_840B = train_model(glove_840B_embeddings)

In [ ]:
model_w2v = train_model(w2v)

In [ ]:
# Gender by Name. (2020). UCI Machine Learning Repository. https://doi.org/10.24432/C55G7X.

gender_names = pd.read_csv("data/gender_bias_test_data/uci_gendered_names/name_gender_dataset.csv")

gender_names.info()

In [ ]:
num_f = gender_names[gender_names.Gender == 'F'].shape[0]
num_m = gender_names[gender_names.Gender == 'M'].shape[0]

print('There are {} female instants vs {} male instants'.format(num_f, num_m))

In [ ]:
GENDER_CAT_NAMES = {
    'female': gender_names[gender_names.Gender == 'F'].iloc[:num_m].Name.values,
    'male': gender_names[gender_names.Gender == 'M'].Name.values
}

In [ ]:
def cat_sentiment(model, embeddings, group_data, isText=False, swarmPlot=True):
    print("---- Get the sentiment table")
    # get the name table of different people's names and save embeddings
    category_sentiments = category_sentiment_table(model, embeddings, group_data, isText)
    
    stats_params = []
    for group in category_sentiments.group.unique():
        stats_params.append(category_sentiments['sentiment'][category_sentiments['group'] == group])
    
    print("---- Calculate F_oneway")
    stats.f_oneway(*stats_params)
    fstat,pval = stats.f_oneway(*stats_params)
    print('F-statistic:',fstat,' With P-value:', pval)

    # Show the results on a swarm plot, with a consistent Y-axis
    matplotlib.pyplot.figure(figsize=(15,5))
    matplotlib.pyplot.subplot(121)
    plot = None
    if swarmPlot:
        plot = seaborn.swarmplot(x='group', y='sentiment', data=category_sentiments)
    else:
        plot = seaborn.violinplot(x='group', y='sentiment', data=category_sentiments)
    plot.set_ylim([-10, 10])
    matplotlib.pyplot.subplot(122)
    plot = seaborn.barplot(x='group', y='sentiment', data=category_sentiments, capsize=.1)
    matplotlib.pyplot.suptitle("hehe", fontsize=16)

In [ ]:
cat_sentiment(model_840B ,glove_840B_embeddings, GENDER_CAT_NAMES, False, False)

In [ ]:
cat_sentiment(model_w2v ,w2v, GENDER_CAT_NAMES, False, False)

In [ ]:
# Gendered Words: https://github.com/ecmonsen/gendered_words

gendered_words = pd.read_json('data/gender_bias_test_data/gendred_words/gendred_words.json')

gendered_words.info()

In [ ]:
gendered_words = gendered_words.drop('gender_map', axis=1)
gendered_words = gendered_words.drop('wordnet_senseno', axis=1)

In [ ]:
gendered_words.info()

In [ ]:
num_f_2 = gendered_words[gendered_words.gender == 'f'].shape[0]
num_m_2 = gendered_words[gendered_words.gender == 'm'].shape[0]

print('There are {} female instants vs {} male instants'.format(num_f_2, num_m_2))

In [ ]:
GENDER_CAT_NAMES_2 = {
    'female': gendered_words[gendered_words.gender == 'f'].iloc[:num_f_2].word.values,
    'male': gendered_words[gendered_words.gender == 'm'].iloc[:num_f_2].word.values
}

In [ ]:
cat_sentiment(model_840B ,glove_840B_embeddings, GENDER_CAT_NAMES_2)

In [ ]:
cat_sentiment(model_w2v ,w2v, GENDER_CAT_NAMES_2)

In [ ]:
# Google Case study for gender bias translations
# https://ai.google/static/documents/case-study-translate-gender-bias.pdf

google_translation = pd.read_csv("data/gender_bias_test_data/google_translation_gender_bias/google_en_es_gender_bias.csv")

In [ ]:
google_translation.info()

In [ ]:
google_translation = google_translation.drop(
    [
        'sourceLanguage', 'targetLanguage', 'documentID', 'stringID',
        'translatedText', 'entityName', 'sourceURL'
    ],
    axis=1)

df['short_str'] = df['long_str'].str.slice(0,3)


In [ ]:
google_translation.info()

In [ ]:
lengths = google_translation.sourceText.str.len()
plot.hist(lengths) 
plot.axis([0,550,0,800])
plot.xlabel('Text Length')
plot.ylabel('Count')
plot.show()

In [ ]:
print("Max length is {}".format(max(lengths)))
print("Min length is {}".format(min(lengths)))
print("Top 10 lengths: ", lengths.value_counts().nlargest(10).to_dict())

In [ ]:
MAX_LENGTH_STR = 513
# Truncate and padd sentences with zeros
google_translation['short_str'] = google_translation['sourceText'].str.slice(0,MAX_LENGTH_STR).str.zfill(MAX_LENGTH_STR)

In [ ]:
google_translation.perceivedGender.unique()

In [ ]:
num_f_3 = google_translation[google_translation.perceivedGender == 'Female'].shape[0]
num_m_3 = google_translation[google_translation.perceivedGender == 'Male'].shape[0]

print('There are {} female instants vs {} male instants'.format(num_f_3, num_m_3))

In [ ]:
NUMBER_SAMPLES = 100
GENDER_CAT_NAMES_3 = {
    'female': google_translation[google_translation.perceivedGender == 'Female'].sample(n=NUMBER_SAMPLES).short_str.values,
    'male': google_translation[google_translation.perceivedGender == 'Male'].sample(n=NUMBER_SAMPLES).short_str.values
}

In [ ]:
%%time
cat_sentiment(model_w2v ,w2v, GENDER_CAT_NAMES_3, True, False)

In [ ]:
%%time
cat_sentiment(model_6B ,glove_6B_300d_embeddings, GENDER_CAT_NAMES_3, True, False)

In [ ]:
%%time
cat_sentiment(model_840B ,glove_840B_embeddings, GENDER_CAT_NAMES_3, True, False)

In [ ]:
# Other Biases to look into:

# religious
# gender
# ageism 
# 

# racial <- is probably the most obvious one